In [ ]:
import sys
import logging

handler = logging.StreamHandler(sys.stdout)
logging.getLogger().setLevel(logging.DEBUG)  # events of this level and above will be tracked
handler.setFormatter(
    logging.Formatter("\n%(asctime)s: %(levelname)s: %(module)s.%(funcName)s:\n%(message)s", "%y-%m-%d %H:%M:%S")
)
logging.getLogger().handlers = [handler,]

In [ ]:
from elfpy.strategies.basic import BasicPolicy

class Shorter(BasicPolicy):
    """
    Agent that is trying to optimize on a rising vault APR via shorts
    """

    def __init__(self, market, rng, wallet_address, budget=10_000):
        """call basic policy init then add custom stuff"""
        self.pt_to_short = 1_000
        super().__init__(market, rng, wallet_address, budget)

    def action(self):
        """
        implement user strategy
        short if you can, only once
        """
        block_position_list = list(self.wallet.token_in_protocol.values())
        has_opened_short = bool(any((x < -1 for x in block_position_list)))
        can_open_short = self.get_max_pt_short() >= self.pt_to_short
        action_list = []
        if can_open_short:
            if self.market.vault_apy > self.market.get_rate():
                action_list.append(self.create_agent_action(
                    action_type="open_short", trade_amount=self.pt_to_short)
                )
            elif self.market.vault_apy < self.market.get_rate():
                if has_opened_short:
                    action_list.append(
                        self.create_agent_action(action_type="close_short", trade_amount=self.pt_to_short)
                    )
        return action_list

In [ ]:
import numpy as np

from elfpy.utils.parse_config import parse_simulation_config
from elfpy.simulators import YieldSimulator

In [ ]:
config = parse_simulation_config(config_dict={
    'title': 'demo simulation config',
    'market': {
        'min_target_liquidity': 1_000_000.0,
        'max_target_liquidity': 10_000_000.0,
        'min_target_volume': 0.001,
        'max_target_volume': 0.01,
        'min_vault_age': 0,
        'max_vault_age': 1,
        'min_vault_apy': 0.001,
        'max_vault_apy': 0.9,
        'base_asset_price': 1
    },
    'amm': {
        'min_fee': 0.1,
        'max_fee': 0.5,
        'min_pool_apy': 0.02,
        'max_pool_apy': 0.9,
        'floor_fee': 0,
        'verbose': False
    },
    'simulator': {
        'num_trading_days': 10,#180,
        'num_blocks_per_day': 100,#7_200,
        'token_duration': 0.2465753424657534,
        'precision': 64,
        'pricing_model_name': 'Hyperdrive',
        'user_policies': [],
        'shuffle_users': True,
        'init_lp': True,
        'random_seed': 123,
        'logging_level': 'info'
    }
})

In [ ]:
simulator = YieldSimulator(config)
simulator.reset_rng(
    np.random.default_rng(simulator.config.simulator.random_seed)
)
simulator.set_random_variables()

ramp = np.linspace(start=0.05, stop=0.1, num=config.simulator.num_trading_days)
override_dict = {
    "pricing_model_name": "HyperDrive",
    "target_liquidity": 10e6,
    "fee_percent": 0.1,
    "init_pool_apy": 0.05,
    "vault_apy": ramp,
    "num_blocks_per_day": 1,  # 1 block a day, keep it fast for testing
}

simulator.setup_simulated_entities(override_dict)
simulator.agents.update(
    {1: Shorter(
            market=simulator.market,
            rng=simulator.rng,
            wallet_address=1
        )
    }
)
simulator.agents.update(
    {2: Shorter(
        market=simulator.market,
        rng=simulator.rng,
        wallet_address=2
    )
    }
)
simulator.run_simulation()

In [ ]:
from elfpy.utils.data import format_trades

[trades, trades_agg] = format_trades(simulator.analysis_dict)

In [ ]:
import matplotlib.pyplot as plt

logging.getLogger().setLevel(logging.WARNING)  # events of this level and above will be tracked

fig, ax = plt.subplots()
ax = trades.plot(x="day", y="vault_apy", ax=ax)
ax = trades.plot(x="day", y="pool_apy", ax=ax)